# f90wrap: an automated tool for constructing deep Python interfaces to modern Fortran codes 

## F2PY


1. declaring compatible data types data types between fortran and python
2. ???error: Unable to find vcvarsall.bat
3. https://www.youtube.com/watch?v=nqsxjwUmmuA

usage

1.  Changes to the code : allocation, subroutine , cmmand strings
2. f2py -m module_name -h sig_file.pyf 
list_of_fortran_files
3. Edit the signature file
4. f2py -c --fcompiler=gnu95 sig_file.pyf list_of_fortran_files



The drawbacks are due to the fact that there is no way for F2PY to determine the actual intention of the arguments; that is, there is ambiguity in distinguishing between input and output arguments. Consequently, F2PY assumes that all arguments are input arguments by default.

next we explore some ways to tell f2py the true intention of function argument

https://numpy.org/devdocs/f2py/f2py.getting-started.html

conduct wrapping process one by one

1. creat signature file 

    python -m numpy.f2py -m fib1 -h fib1.pyf fib1.f 
    1.  -m fib1 module name for python
    2. -h signature file
    3. modify signature file
2.  build the extension module

    1. python -m numpy.f2py -c fib1.pyf fib1.f

3. this approach is suitable for wrapping (e.g. third party) Fortran codes for which modifications to their source codes are not desirable nor even possible.

if we can actually modify the source fortran file, we may use F2py directives. A F2PY directive consists of special comment lines (starting with Cf2py or !f2py, for example) which are ignored by Fortran compilers but interpreted by F2PY as normal lines.
(in fortran , C is the starting sign for commrnt uh)
```
C FILE: FIB3.F
      SUBROUTINE FIB(A,N)
C
C     CALCULATE FIRST N FIBONACCI NUMBERS
C
      INTEGER N
      REAL*8 A(N)
Cf2py intent(in) n
Cf2py intent(out) a
Cf2py depend(n) a
      DO I=1,N
         IF (I.EQ.1) THEN
            A(I) = 0.0D0
         ELSEIF (I.EQ.2) THEN
            A(I) = 1.0D0
         ELSE 
            A(I) = A(I-1) + A(I-2)
         ENDIF
      ENDDO
      END
C END FILE FIB3.F
```


now we knows the basic usage of f2py, we will dig deeper

##### Using f2py as a command-line tool
three modules distinguished by -c -h
###### signature file generation

## ASE

In [2]:
!python N2Cu.py

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 11:48:41       11.689927*       1.0797
BFGSLineSearch:    1[  2] 11:48:41       11.670814*       0.4090
BFGSLineSearch:    2[  4] 11:48:41       11.625880*       0.0409
Adsorption energy: 0.3235194223180411


## F90Wrap

### Intro
#### File based interfaces
the reliance on file-based interfaces requires substantial time investment to maintain a library of input generators and output parsers
#### Deep scripting interfaces
Since modern scientific well-organized codes consist of modules and API functions(not for calculation), we may provide deeper wrapping with modular.

merits
1. middle variable can be called efficiently from memory 
2. easy to debug
3. code bases more sustainable


### Methodology
key steps
1. build abstract symbol tree 
2. remove the not-for-wrap node
3. write a more suitable version of each routine 
4. f2py 
5. write a object-oriented layer for conversion between python object instances and fortran derived-type 

### Usage and features

1. f90wrap -m Module [F90_Files1,F90_files2...] gives
   1. f2py input :f90wrap_F90_FILE.f90 (and f90wrap_toplevel.f90)
   2. python package named MODULE
2. f2py-f90wrap -c -m _MODULE OBJ_FILES f90wrap_∗.f 90 ∗.o LINK_OPTIONS.
    1. present()
    2. python runtimeError
    3. ctrl+c interrupt handler

limitations
1. intent(out) $\to$ intent(inout)
2. To have inout scalars, one can call f90wrap with the-default-to-inout flag and declare the Python variables as single-element numpy arrays (using numpy.zeros(1), for example). 
3. no pointer 
4. no array of derived types

